## Introduction

`calabru` is a model calibration package.

The following example will demonstrate the basic workflow of `calabru`. 

Let's consider a simple beam model created in Python using PyCBA(https://ccaprani.github.io/pycba/notebooks/intro.html) module. We begin with some necessary imports:

In [2]:
import calabru as clb # calabru package
import numpy as np # for numerical arrays
import matplotlib.pyplot as plt # for plotting
from IPython import display # For images in notebook
import pycba as cba # for creating beam model
%matplotlib inline 

The beam model represents a two span bridge, with a UDL of 20 kN/m acting on each span.

In [ ]:
display.Image("./images/intro_cba_1.png",width=800)

For the calibration problem, we wish to update the model such that the simulated deflection at midspan of Span 1, when subjected to the 20kN/m UDL, matches closely with a known target measurement.

In [ ]:
target_deflections = [20] # mm

The model is updated by tuning the bending stiffness parameter (EI).

In [ ]:
EI_start = [30 e6]

### Workflow

In essence, the `calabru` workflow is as follows:

1. Create a function handler for the model that will be updated.

In [ ]:
def create_pycba(parameters):
    # init parameters according to PyCBA
    EI = paramter[0]
    LM = [[1, 1,  20, 0, 0], [2, 1, 20, 0, 0]]
    L = [7.5, 7.0]
    R = [-1, 0, -1, 0, -1, 0]
    
    # PyCBA commands
    beam_analysis = cba.BeamAnalysis(L, EI, R, LM)
    beam_analysis.analyze()
    # Optional plotting of results
    beam_analysis.plot_results()

    deflection_responses = [beam_analysis.beam_results.vRes[1].M.max()]
    
    return deflection_responses

As can be seen, `create_pycba` is a function handler 
which creates and analyzes the PyCBA beam model. The function handler must have the following feature:
- Updating parameters as a list/array.
- returns the model output that corresponds to the target measurements as a list/array.

For this example, `parameters` is the list input argument of the bending stiffness while `deflection_responses` is the list of deflection output from the beam model.

2. Invoke the `ModelUpdating` class.

In [ ]:
update_process = clb.ModelUpdating(
    function_handle=create_pycba, 
    param_list=EI_start,
    target_list=target_deflections
)

3. Run the class function to update model.

In [ ]:
update_process.update_model()

For results, we can directly access the class variables of `update_process` (primarily list/arrays). As such, the parameters and residuls can be use for post-processing later on (e.g., plotting).

In [ ]:
# for update history of parameters
update_process.param_update_history 
# for residues
update_process.residue_history 
# for history of model differences
update_process.resp_diff_history

The typical outputs from the above command looks something like: